# ECON 490: Good Regression Practice (13)

## Prerequisites

1. Importing data into R.
2. Creating new variables in R.
3. Running OLS regressions.

## Learning Outcomes

1. Identify and correct for outliers by trimming or winsorizing the dependent variable. 
2. Identify and correct for the problem of multicollinearity.
3. Identify and correct for the problem of heteroskedasticity. 
4. Identify and correct for the problem of non-linearity. 

## 13.1 Dealing with Outliers 

Imagine that we have constructed a dependent variable which contains the earnings growth of individual workers and we see that some worker's earnings increased by more than 400%. We might wonder if this massive change is just a coding error made by the statisticians that produced the data set. Even without that type of error, though, we might worry that the earnings growth of a small number of observations are driving the results of our analysis. If this is the case, we are producing an inaccurate analysis based on results that are not associated with the majority of our observations. 

The standard practice in these cases is to either winsorize or trim the subset of observations that are used in that regression. Both practices remove the outlier values in the dependent variable to allow us to produce a more accurate empirical analysis. 

<div class="alert alert-block alert-warning">
    
<b>Warning:</b> You should only consider fixing outliers when there is a clear reason to address this issue. Do not apply the tools below if the summary statistics in your data make sense to you in terms of abnormal values. For example, outliers might be a sign that your dependent and independent variables have a non-linear relationship. If that is the case, you will want to consider including an interaction term that addresses that non-linearity. 
    
</div>

#### 13.1.1 Winsorizing a dependent variable

Winsorizing is the process of limiting the extreme values of a dependent variable to reduce the effect of (possibly erroneous) outliers. It consists of replacing values below the $a$ percentile with that percentile value and values above the $b$ percentile with that percentile value. Consider the following example using our fake data set:

In [ ]:
#Clear the memory from any pre-existing objects
rm(list=ls())

# loading in our packages
library(tidyverse) #This includes ggplot2! 
library(haven)
library(IRdisplay)

#Open the dataset 
fake_data <- read_dta("../econ490-stata/fake_data.dta")  


In [ ]:
quantile(fake_data$earnings, probs = c(0.01,0.99))

In [ ]:
min(fake_data$earnings)

In [ ]:
max(fake_data$earnings)

From the summary statistics above, we can see that that the income earned by the individual at the 1st percentile is 2,831.03 and that the lowest earner in the data set earned 8.88.  

We can also see that income earned by the individual at the 99th percentile is only 607,140.32 and that the highest earner in the data earned over 60 millions!

These facts suggest to us that there are large outliers in our dependent variable.

We want to get rid of these outliers by winsorizing our data set. What that will mean is replacing the earnings of all observations below the 1st percentile by exactly the earnings of the individual at the 1st percentile. Further, it means replacing the earnings of all observations above the 99th percentile by exactly the earnings of the individual at the 99th percentile. 

To winsorize this data, we do the following:

1. Create a new variable using the command `mutate` with the same values as earnings that we call *log_earnings_winsor*. We decide to store the winsorized version of the dependent variable as a different variable for organizational purposes - we don't want to overwrite the original data set.
2. If earnings are smaller than the 1st percentile, we replace the values of *log_earnings_winsor* with the earnings of the individual at the 1st percentile: `(quantile(fake_data$earnings, probs = 0.01) = 2831)`.
3. If earnings are larger than the 1st percentile, we replace the values of *earnings_winsor* with the earnings of the individual at the 99th percentile: `(quantile(fake_data$earnings, probs = 0.99) = 607140 )`.

The values of this new variable will be created using the command `ifelse()`. If earnings are less than 2831, the value of *log_earnings_winsor* is replaced by the log(2831) using this command.

You can run these commands yourself below:

In [ ]:
fake_data <- fake_data %>%
        mutate(log_earnings_winsor = ifelse(earnings<2831 ,  log(2831),  ifelse( earnings  > 607140 , log(607140), log(earnings)))) 


Now we can use this new dependent variable in our regression analysis. If the outliers were not creating problems, there will be no change in the results. If they were creating problems, those problems will now be fixed. 

Let's take a look at this by first running the regression from [Module 10](econometrics/econ490-r/10_Linear_Reg.ipynb) with the original earning variable. 

In [ ]:
lm(data=fake_data, log(earnings) ~ as.factor(sex))


In [ ]:
lm(data=fake_data, log_earnings_winsor ~ as.factor(sex))


Do you think that in this case the outliers were having a significant impact before being winsorized?

#### 13.1.2 Trimming a dependent variable 

Trimming consists of replacing both values below the $a$ percentile and values above the $b$ percentile by a missing value. This is done to exclude these outliers from regression, since R by design does not include missing (`NA`) observations in the `lm` command.

Below we trim our earnings variable. 

In [ ]:
fake_data <- fake_data %>%
        mutate(log_earnings_trimmed = ifelse(earnings<2831 ,  NA,  ifelse( earnings  > 607140 , NA, log(earnings)))) 


Here is the result of the regression with the new dependent variable. 

In [ ]:
lm(data=fake_data, log_earnings_trimmed ~ as.factor(sex))

## 13.2 Multicollinearity 

If two variables are linear combinations of one another they are multicollinear. Ultimately, R does not allow us to include two variables in a regression that are perfect linear combinations of one another, such as a constant or a dummy variable for male and a dummy for female (since female = 1 - male). In all of the regressions above, we see that one of those variables was dropped from the regression "because of collinearity".

In [ ]:
fake_data <- fake_data %>%
        mutate(male = case_when(sex == 'M' ~ 1, sex == 'F' ~ 0)) %>%
        mutate(female = case_when(sex == 'F' ~ 1, sex == 'M' ~ 0))

In [ ]:
lm(data=fake_data, log_earnings_trimmed ~ male + female)

Is this a problem? Not really. Multicollinearity is a sign that a variable is not adding any new information. Notice that with the constant term and a male dummy we can know the mean earnings of females. In this case, the constant term is by construction the mean earnings of females, and the male dummy gives the earning premium for male workers.

While there are some statistical tests for multicollinearity, nothing beats having the right intuition when running a regression. If it is clear that two variables contain essentially the same information, we should avoid including both in our analysis. For instance, we might have an age variable that includes both years and months (e.g. if a baby is 1 year and 1 month old, then this age variable would be coded as 1 + 1/12 = 1.083). If we include this variable in a regression that also includes an age variable for only years (e.g the baby's age would be coded as 1) then we would have the problem of multicollinearity. Because they are not perfectly collinear, R might still produce some results, but the coefficients on these two variables would be biased. 

## 13.3 Heteroskedasticity 

When we run a linear regression, we essentially split the outcome into a (linear) part explained by observables and an error term:

$$
y_i = a + b x_i + e_i
$$ 

The standard errors in our coefficients depend on $e_i^2$ (as you might remember from ECON 326). Heteroskedasticity refers to the case in which the variance of this projection error depends on the observables $x_i$. For instance, the variance of wages tends to be higher for people who are university educated (i.e. there are some people with very high wages) whereas it is small for people who are non-university educated (i.e. they tend to be concentrated in smaller paying jobs). R by default assumes that the variance does not depend on the observables, which is known as homoskedasticity. It is safe to say that this is an incredibly restrictive assumption.

While there are tests for heteroskedasticity, the more empirical economists rely on including heteroskedastic consistent standard errors as a default in their regressions. The most standard way to do this is to use `feols`, another command similar to `lm()` that comes from the `fixest` package.

In [ ]:
install.packages('fixest')
library(fixest)

In [ ]:
model = feols(log_earnings_trimmed ~ as.factor(sex) , fake_data)


In [ ]:
summary(model, vcov="HC1")

A best practice is simply to always use robust standard errors in your ECON 490 project.

## 13.4 Non-linearity

Our regression analysis so far assumes that the relationship between our explained and explanatory variables is linear. If this is not the case, meaning the relationship is non-linear, then we will get inaccurate results from our analysis. 

Let's consider an example. We know that earnings increases with age, but what if economic theory predicts that the amount at which earnings increase for each year of age when workers are younger is larger than the amount at which earnings increase for each year of age when workers are older? What we are asking here is whether earnings is increasing with age at a decreasing rate. In essence, we want to check whether there is a concave relation between age and earnings. We can think of several mechanisms for why this relationship might exist: for a young worker, as they age they get higher wages through increased experience in the job; for an older worker, as they age those wage increases will be smaller as there are smaller productity gains with each additional year working. In fact, if the productivity of workers decreaseas as they age, perhaps for reasons related to health, then it is possible to find a negative relationship between age and earning beyond a certain age; the relationship is an inverted U-shape. 

We can correct for this in our model by including another variable, that is age to the power of two.

In [ ]:
fake_data <- fake_data %>% mutate(age2 = age^2) 

In [ ]:
model = lm(log_earnings_trimmed ~ age + age2, fake_data)

In [ ]:
summary(model, vcov="HC1")

There does seem to be some evidence in our regression results that this economic theory is correct, since the coefficient on the interaction term is both negative and statistically significant.

How do we interpret these results? Let's think about the equation we have just estimated:
$$
Earnings_i = \beta_0 + \beta_1 Age_i + \beta_2 Age^2_i + \varepsilon_i 
$$
This means that earnings of an individual change in the following way with their age:
$$
\frac{\partial Earnings_i}{\partial Age_i} = \beta_1 + 2 \beta_2 Age_i
$$
Due to the quadratic term, as age changes, the relationship between age and earnings changes as well.
We have just estimated $\beta_1$ to be positive and equal to 0.067, and $\beta_2$ to be negative and equal to 0.001.
This means that as age increases, its correlation with earnings decrease:
$$
\frac{\partial Earnings_i}{\partial Age_i} = 0.067 - 0.002 Age_i
$$


Since the marginal effect changes with the size of $Age_i$, providing one unique number for the marginal effect becomes difficult.
The most frequently reported version of this effect is the ''marginal effect at the means'': the marginal effect of age on earnings when age takes its average value.
In our case, this will be equal to 0.067 minus 0.002 times the average value of age.


In [ ]:
mean(fake_data$age)

Since the average age of workers is 45, the marginal effect of age at the means is 
$$
0.067 - 2 * 0.001 * 45 = -0.023
$$
What does that mean? It means that, for the average person, becoming one year older is associated with a 2% decrease in log earnings.

Notice that this is the effect for the *average person*. Is the same true for young workers and elder workers? To learn how to interpret this non-linearity in age, let's see how the predicted earnings correlate with age.
We can obtain the predicted earnings with the `predict` function and then use a scatterplot to eyeball its relationship with age. 

Let's see how to do this step by step.
First, we store our regression in the object *fit*. Second, we use the function `model.frame` to keep only the observations and variables used in our regression. Then, we use `predict` to store in data frame *yhat* the predicted earnings obtained from our regression. Notice that `predict` creates a list, therefore we transform it into a dataframe using the function `as.data.frame`. Finally, we merge the two dataframes together using the function `cbind`.

In [ ]:
# Run the regression with the quadratic term
fit = lm(log_earnings_trimmed ~ age + age2, fake_data)

# Predict earnigns and save them as yhat in the same dataframe
datareg <- model.frame(fit)          # keep observations used in regression
yhat <- as.data.frame(predict(fit))  # save predicted earnings as data frame
datareg = cbind(datareg, yhat)       # merge the two dataframes
datareg <- datareg %>% rename(yhat = "predict(fit)") # rename

Once we have all the information in one unique dataframe called *datareg*, we can display a scatterplot with age on the x-axis and predicted log-earnings on the y-axis. We covered how to create scatterplots in [Module 8](econometrics/econ490-r/8_ggplot_graphs.ipynb).

In [ ]:
# Create scatterplot
ggplot(data = datareg, aes(x=age, y=yhat)) + geom_point()

The scatterplot shows an inverted-U relationship between age and the predicted log-earnings. This relationship implies that, when a worker is very young, becoming older is positively correlated with earnings. However, after a certain age, this correlation becomes negative and the worker gets lower earnings for each additional year of age. In fact, based on this graph workers earns start to decline just after the age of 50. Had we modelled this as a linear model we would have missed this important piece of information! 

<div class="alert alert-info">


**Note:** If there is a theoretical reason for believing that non-linearity exists, R provides some tests for non-linearity. You can also create a scatter plot to see if you can observe a non-linear relationship in the data. We covered that approach in [Module 9](econometrics/econ490-r/9_ggplot_graphs.ipynb).
    
</div>

## 13.5 Wrap Up 

While it is important to always follow best practices for regression analysis, such as checking and correcting for outliers, heteroskedaticity, multicollinearity and non-linearity, this can be more of an art than a science. If you need any guidance on whether or not you need to address these issues, please speak with your instructor or TA.